In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import sinabs
from torchvision import transforms
from PIL import Image
import sinabs.layers as sl
import numpy as np
import quartz
import copy
from tqdm.auto import tqdm
from quartz.utils import get_accuracy, encode_inputs, decode_outputs, plot_output_histograms, plot_output_comparison, normalize_weights
from typing import List

np.set_printoptions(suppress=True)

In [ ]:
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [ ]:
imagenet = torchvision.datasets.ImageNet('/home/gregorlenz/Development/playground/data/ImageNet/', split='val', transform=preprocess)

In [ ]:
testloader = torch.utils.data.DataLoader(imagenet, batch_size=128, shuffle=True, num_workers=4)

In [ ]:
device = "cuda"
cpu = "cpu"

model = models.vgg11(weights=models.vgg.VGG11_Weights.DEFAULT)
# model = models.vgg11_bn(weights=models.vgg.VGG11_BN_Weights.DEFAULT)

In [ ]:
layer1 = nn.Conv2d(3, 3, kernel_size=1, groups=3)
layer1.weight.data /= layer1.weight.data
layer1.bias = torch.nn.Parameter(-1*torch.tensor([0.485, 0.456, 0.406]))

layer2 = nn.Conv2d(3, 3, kernel_size=1, groups=3)
layer2.weight = nn.Parameter(1/torch.tensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(1).unsqueeze(1))
layer2.bias.data -= layer2.bias.data

model = nn.Sequential(layer1, layer2, *model.features, model.avgpool, nn.Flatten(), *model.classifier[0:2], *model.classifier[3:5], model.classifier[-1]) #*model.classifier)
model = model.eval()

for layer in list(model.children())[2:]:
    if isinstance(layer, nn.ReLU):
        layer.inplace = False

In [ ]:
# model

In [ ]:
# get_accuracy(model, testloader, device)

In [ ]:
norm_model = copy.deepcopy(model)
norm_model = norm_model.eval()

In [ ]:
param_layer_names = [name for name, child in norm_model.named_children() if isinstance(child, (nn.Conv2d, nn.Linear))][2:]
output_layer_names = [name for name, child in norm_model.named_children() if isinstance(child, nn.ReLU)]
output_layer_names += [param_layer_names[-1]]
# param_layers = ['2', '5', '8', '10', '13']
percentile = 99.99

print(param_layer_names)

In [ ]:
normloader = torch.utils.data.DataLoader(imagenet, batch_size=150, shuffle=True, num_workers=0)
images, labels = next(iter(normloader))

In [ ]:
normalize_weights(norm_model.to(device), images.to(device), param_layer_names=param_layer_names, percentile=percentile)

In [ ]:
plot_output_comparison(model.to(cpu), norm_model.to(cpu), images.to(cpu), output_layers=param_layer_names, every_n=10000, every_c=10, savefig=f"norm_activation_correct_biases.png")

In [ ]:
sinabs.utils.normalize_weights(norm_model.to(device), images.to(device), param_layers=param_layer_names, output_layers=output_layer_names, percentile=percentile)

In [ ]:
get_accuracy(norm_model, testloader, device=device)#"cpu")

In [ ]:
snnloader = torch.utils.data.DataLoader(imagenet, batch_size=2, shuffle=True, num_workers=4)

accuracies = []
# for exponent in range(5, 7):
exponent = 5
t_max = 2**exponent
snn = quartz.from_torch.from_model(norm_model, t_max=t_max, add_spiking_output=True).eval()
snn = snn[2:] # nn.Sequential(*snn[:2], quartz.IF(t_max=t_max, rectification=False), *

In [ ]:
assert 1 == 0;

In [ ]:
snn = snn.cpu()
norm_model = norm_model.cpu()

torch.random.manual_seed(6)
# snn[1].module.weight.data /= 2

# images, labels = next(iter(snnloader))
for images, labels in iter(snnloader):
    norm_images = norm_model[:2](images)
    temp_images = encode_inputs(norm_images, t_max=t_max)
    conv_output = snn(temp_images)
    snn_output = decode_outputs(conv_output, t_max=t_max)
    print(snn_output.argmax(1) == labels)

ann_output = norm_model(images)

In [ ]:
snn[1].early_spikes

In [ ]:
ann_output.argmax(1)

In [ ]:
snn_output.argmax(1)

In [ ]:
accuracy = get_accuracy(snn, testloader, device="cpu", t_max=t_max)
accuracies.append(accuracy)
print(f"{t_max} time steps: {round(accuracy, 3)}%")

In [ ]:
snn

In [ ]:
snn

In [ ]:
with torch.no_grad():
    output = ann(input_batch.cpu())

probabilities = torch.nn.functional.softmax(output[0], dim=0)
print_probabilities(probabilities)

In [ ]:
snn = quartz.from_torch.from_model(ann, t_max=t_max, batch_size=batch_size)

In [ ]:
temp_q_values = quartz.encode_inputs(input_batch, t_max=t_max)

In [ ]:
# snn = snn.cuda()
# temp_q_values = temp_q_values.cuda()

In [ ]:
with torch.no_grad():
    temp_output = snn(temp_q_values.flatten(0, 1)).unflatten(0, (batch_size, -1))
snn_output = quartz.decode_outputs(temp_output, t_max=t_max)

In [ ]:
probabilities = torch.nn.functional.softmax(snn_output[0], dim=0)
print_probabilities(probabilities)

In [ ]:
snn_output.shape

In [ ]:
# torchvision.datasets.ImageNet("../data", split="val")

In [ ]:
snn_output